In [1]:
import pythainlp
from pythainlp import sent_tokenize, word_tokenize
from pythainlp import word_tokenize, Tokenizer
from pythainlp.util.trie import Trie, dict_trie
from pythainlp.corpus.common import thai_words
from pythainlp.util import normalize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import word_tokenize
import pandas as pd
import json

In [2]:
with open("custom_words.json", encoding='utf-8') as w:
    MY_CUSTOM_WORDS = json.load(w)

In [3]:
df = pd.read_excel('input/vendor_data.xlsx')
df.head()

,ProductCode,ProductName,Qty,Pack,Weight
0,1101155228,วิกซอล โกลด์ 900 (2+1),12,1,12.5
1,1102043121,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,12,1,9.8
2,1201033318,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,12,1,8.5
3,1201053312,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,12,1,8.5
4,1201133155,วิช-ถูพื้น เนเจอร์ ชมพู 5200,4,1,21.9


In [4]:
text = "ไฮยีนปรับเข้มข้นไลฟ์ มิราเคิลโรสโกล115"
print("sent_tokenize:", sent_tokenize(text))
print("word_tokenize:", word_tokenize(text))
print("no whitespace:", word_tokenize(text, keep_whitespace=False))

sent_tokenize: ['ไฮยีนปรับเข้มข้นไลฟ์ มิราเคิลโรสโกล115']
word_tokenize: ['ไฮ', 'ยีน', 'ปรับ', 'เข้มข้น', 'ไลฟ์', ' ', 'มิราเคิล', 'โร', 'ส', 'โก', 'ล', '115']
no whitespace: ['ไฮ', 'ยีน', 'ปรับ', 'เข้มข้น', 'ไลฟ์', 'มิราเคิล', 'โร', 'ส', 'โก', 'ล', '115']


In [5]:
#words = MY_CUSTOM_WORDS
words = []
custom_words_list = set(thai_words())
## add multiple words
custom_words_list.update(words)

## add word
# custom_words_list.add('เป็งปุ๊ด')
# custom_words_list.add('เพ็ญพุธ')
trie = dict_trie(dict_source=custom_words_list)

custom_tokenizer = Tokenizer(custom_dict=trie, engine='newmm')

print("custom :", custom_tokenizer.word_tokenize(text))

custom : ['ไฮ', 'ยีน', 'ปรับ', 'เข้มข้น', 'ไลฟ์', ' ', 'มิราเคิล', 'โร', 'ส', 'โก', 'ล', '115']


In [6]:
stopwords = []
stopwords.append("(")
stopwords.append("-")
stopwords.append("_")
stopwords.append("")
stopwords.append(" ")
stopwords.append(")")

stopwords

['(', '-', '_', '', ' ', ')']

In [7]:
train = df.copy()

In [8]:
train.head()

,ProductCode,ProductName,Qty,Pack,Weight
0,1101155228,วิกซอล โกลด์ 900 (2+1),12,1,12.5
1,1102043121,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,12,1,9.8
2,1201033318,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,12,1,8.5
3,1201053312,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,12,1,8.5
4,1201133155,วิช-ถูพื้น เนเจอร์ ชมพู 5200,4,1,21.9


In [9]:
# Normalize text
train['ProductName_clean1'] = train.apply(lambda row: normalize(row['ProductName']), axis=1)
 
# Word segmentation: it will take a while....
train['ProductName_clean2'] = train.apply(lambda row: custom_tokenizer.word_tokenize(row['ProductName_clean1']), axis=1)
 
# Remove stopwords, it will take a very long time.....
train['ProductName_clean3'] = train.apply(lambda row: " ".join([i for i in row['ProductName_clean2'] if i not in stopwords]), axis=1)
 
# Removing the 'TutorialsPoint' from the list using the remove() method
def remove_stopwords(input_list, stopwords=stopwords):
    return [word for word in input_list if word not in stopwords]

train['ProductName_clean4'] = train.apply(lambda row: [word for word in row['ProductName_clean2'] if word not in stopwords], axis=1)


In [10]:
train.head()

,ProductCode,ProductName,Qty,Pack,Weight,ProductName_clean1,ProductName_clean2,ProductName_clean3,ProductName_clean4
0,1101155228,วิกซอล โกลด์ 900 (2+1),12,1,12.5,วิกซอล โกลด์ 900 (2+1),"[วิก, ซอล, , โกลด์, , 900, , (2+1)]",วิก ซอล โกลด์ 900 (2+1),"[วิก, ซอล, โกลด์, 900, (2+1)]"
1,1102043121,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,12,1,9.8,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,"[วิก, ซอล, , ออก, ซี่, , มิ้นท์, , 700, , ...",วิก ซอล ออก ซี่ มิ้นท์ 700 มล. 3 S,"[วิก, ซอล, ออก, ซี่, มิ้นท์, 700, มล., 3, S]"
2,1201033318,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,"[วิ, ช, -, ถู, พื้น, , ท, รี, ดี, , ชมพู, ,...",วิ ช ถู พื้น ท รี ดี ชมพู 650 ซอง,"[วิ, ช, ถู, พื้น, ท, รี, ดี, ชมพู, 650, ซอง]"
3,1201053312,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,"[วิ, ช, -, ถู, พื้น, , ท, รี, ดี, , ฟ้า, , ...",วิ ช ถู พื้น ท รี ดี ฟ้า 650 ซอง,"[วิ, ช, ถู, พื้น, ท, รี, ดี, ฟ้า, 650, ซอง]"
4,1201133155,วิช-ถูพื้น เนเจอร์ ชมพู 5200,4,1,21.9,วิช-ถูพื้น เนเจอร์ ชมพู 5200,"[วิ, ช, -, ถู, พื้น, , เนเจอร์, , ชมพู, , 5...",วิ ช ถู พื้น เนเจอร์ ชมพู 5200,"[วิ, ช, ถู, พื้น, เนเจอร์, ชมพู, 5200]"


In [11]:
train.to_excel("input/tokenized_data2.xlsx", index=False)